In [1]:
%%configure -f
{ "conf": {"spark.jars.packages": "com.databricks:spark-avro_2.11:4.0.0" }}

In [3]:
from pyspark.sql import SparkSession
import re

spark = SparkSession \
    .builder \
    .appName("spark-avro-json-sample") \
    .config('spark.hadoop.avro.mapred.ignore.inputs.without.extension', 'false') \
    .getOrCreate()


#main code

path = "wasbs://github@renjie.blob.core.chinacloudapi.cn/renjie/renjieeh/*/2019/05/*/*/*/46.avro"
avroDf = spark.read.format("com.databricks.spark.avro").option("mergeSchema", "true").load(path)
        
#avro->json
jsonRdd = avroDf.select(avroDf.Body.cast("string")).rdd.map(lambda x: ("[" + re.sub("}\n{\"id\"", "},{\"id\"", str(x[0])) + "]"))
datax = spark.read.json(jsonRdd)



datawatch = datax.filter("type == 'WatchEvent'").groupby("Repo.name").count() 
dataevent = datax.filter("type == 'IssuesEvent' OR type == 'IssueCommentEvent'").groupby("Repo.name").count()

datajoin = datawatch.selectExpr("name as name", "count as countwatch")
dataevent = dataevent.selectExpr("name as name", "count as countevent")


datajoin = datajoin.join(dataevent, "name", 'outer')



from pyspark.sql.functions import udf,col
from pyspark.sql.types import LongType
def func(s,a):
    if s != None:
        s = s * 3
    elif s == None:
        s = 0
    else:
        s = 0
    
    if a != None:
        a = a * 7
    elif a == None:
        a = 0
    else:
        a = 0
    
    r = a + s
    return r
my_func = udf(func,LongType())
datajoin = datajoin.withColumn('index', my_func('countwatch','countevent'))


datax = datajoin.sort("index", ascending=False)


#Plot barchart for popularity index

import pandas as pd
import matplotlib.pyplot as plt

dataplot = datax.select("name", "index").collect()
name = [item[0] for item in dataplot]
index = [item[1] for item in dataplot]
dataplot = {"name" : name, "index": index}
dataplot = pd.DataFrame(dataplot)
dataplot = dataplot.iloc[:20]

myplot = dataplot.plot(figsize = (20,20), kind = "barh", color = "#b35900", width = 0.8,
                               x = "name", y = "index", legend = False)

myplot.invert_yaxis()
plt.xlabel("Number of Project", fontsize = 28)
plt.ylabel("Name of Project", fontsize = 28)
plt.title("Name of Project by number", fontsize = 36)
plt.xticks(size = 24)
plt.yticks(size = 24)

plt.show()
